# Quick Start

In this part, we provide some basic yet all-in-one examples to show the features of Torch-Pruning. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.path.abspath("../"))

import torch
from torchvision.models import resnet18
import torch_pruning as tp

### Method 1. Pruning with DepGraph

``DependencyGraph`` serves as the cornerstone of Torch-Pruning, which automatically identifies and groups all layers with inter-dependency. In structural pruning, two layers with dependency should be pruned simultaneously. Therefore, to prune a complicated model, we need to handle those layers carefully. The following example shows the pipeline of pruning single layer in a ResNet-18.

In [ ]:
# 0. prepare your model and example inputs
model = resnet18(pretrained=True).eval()
example_inputs = torch.randn(1,3,224,224)

# 1. build dependency graph for resnet18
DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)

# 2. Select some channels to prune. Here we prune the channels indexed by [2, 6, 9].
pruning_idxs = pruning_idxs=[2, 6, 9]
pruning_group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=pruning_idxs )

# 3. prune all grouped layer that is coupled with model.conv1
if DG.check_pruning_group(pruning_group):
    pruning_group.prune()

After invoking the ``.exec`` method, an inplace pruning will be applied to the model. Upon printing the model, we can notice that multiple layers, such as "model.conv1", "model.bn1", and "model.layer1[0].conv1" are pruned by Torch-Pruning.

In [ ]:
print("After pruning:")
print(model)

Let's inspect the pruning group. The results will show how a pruning operation triggers (=>) another one.

In [ ]:
print(pruning_group)

You can also get all groups from ``DependencyGraph`` using the ``get_all_groups`` method. 

In [ ]:
all_groups = list(DG.get_all_groups())
print("Number of Groups: %d"%len(all_groups))
print("The last Group:", all_groups[-1])

---

### 2. High-level Pruners

Pruning a neural network using the ``DependencyGraph`` can be still complicated, especially for models with numerous layers. Therefore, we also offer high-level pruners to simplify this process. For example, you can easily prune a ResNet18 model with a simple magnitude-based pruner. This method removes weights with small magnitude in the network, resulting in a smaller and faster model without too much performance lost in accuracy

In [43]:
# Instantiate the Hopenet model
from hopenet import Hopenet
import torchvision
import copy
Hnet = torchvision.models.mobilenet_v3_large()
Hnet.classifier = torch.nn.Linear(960, 2, bias=False)
saved_state_dict = torch.load(r"C:\Users\Girish\Desktop\Project2_faceVSnonface\Results_MaskNonmask\MobileNetv3L_MaskNonmask_7\MobileNetv3L_MaskNonmask_7.pt")
Hnet.load_state_dict(saved_state_dict)
model = copy.deepcopy(Hnet)

In [44]:
# model = resnet18(pretrained=True)
example_inputs = torch.randn(1, 3, 112, 112)

# 0. importance criterion for parameter selections
imp = tp.importance.MagnitudeImportance(p=2, group_reduction='mean')

# 1. ignore some layers that should not be pruned, e.g., the final classifier layer.
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear):
        ignored_layers.append(m) # DO NOT prune the final classifier!

        
# 2. Pruner initialization
iterative_steps = 5 # You can prune your model to the target pruning ratio iteratively.
pruner = tp.pruner.MagnitudePruner(
    model, 
    example_inputs, 
    global_pruning=False, # If False, a uniform ratio will be assigned to different layers.
    importance=imp, # importance criterion for parameter selection
    iterative_steps=iterative_steps, # the number of iterations to achieve target ratio
    pruning_ratio=0.6, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers,
)

base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
for i in range(iterative_steps):
    # 3. the pruner.step will remove some channels from the model with least importance
    pruner.step()
    
    # 4. Do whatever you like here, such as fintuning
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    print(model)
    # print(model(example_inputs).shape)
    print(
        "  Iter %d/%d, Params: %.2f M => %.2f M"
        % (i+1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
    )
    print(
        "  Iter %d/%d, MACs: %.2f G => %.2f G"
        % (i+1, iterative_steps, base_macs / 1e9, macs / 1e9)
    )
    # finetune your model here
    # finetune(model)
    # ...



MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 14, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(14, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=14, bias=False)
          (1): BatchNorm2d(14, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(14, 14, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(14, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(14, 56, kernel_size=(1, 1), stride=(1, 1), bi

In [45]:
import torch.nn as nn
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements
def get_model_size(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB
print(f"Model : {get_model_size(model)/MiB:.2f}")


Model : 1.92


In [46]:
# import torch
# # print(model)
# torch.save(model, "C:\\Users\\Girish\\Desktop\\Project2_faceVSnonface\\PrunedModels\\MaskNonmask@06.pt")

# Thresholding and Separating

In [14]:
import numpy as np
import pandas as pd

file = pd.read_csv("C:\\Users\\Girish\\Desktop\\FF-Projects\\deep-head-pose\\output\\aug5Faces.csv")
file

,Name,Infer_Time,Yaw,Pitch,Roll
0,Face\0_1.jpg,26.505709,0.398987,-9.205559,0.017509
1,Face\0_10.jpg,5.069017,7.803657,-0.434235,-2.734573
2,Face\0_100.jpg,5.005598,11.265129,-14.397903,-4.169159
3,Face\0_1000.jpg,0.000000,-10.555687,-18.525948,4.746704
4,Face\0_10000.jpg,5.004644,2.976860,-12.777626,-2.155907
...,...,...,...,...,...
60569,Face\3_landmk_Jesse_Jackson_0001_aug_rotate_0.png,15.620947,-5.690567,2.415878,-23.908890
60570,Face\3_landmk_Jesse_Jackson_0001_aug_rotate_1.png,0.000000,-3.697693,-6.859383,14.106476
60571,Face\4_17850_cloth.jpg,0.000000,3.707451,-25.136688,6.364319
60572,Face\4_17953_cloth.jpg,17.097950,-3.144951,-28.685059,7.699142


In [19]:
# highPose = file[(abs(file.iloc[:, 2:]) > 30).any(axis=1)]
# highPose = file[(abs(file['Yaw']) < 30) & (abs(file['Pitch']) < 15)]
highPose = file[(abs(file['Yaw']) > 60)]
highPose

,Name,Infer_Time,Yaw,Pitch,Roll
30,Face\0_10038.jpg,9.006262,-70.151840,-4.632065,12.673988
45,Face\0_10052.jpg,5.030870,-60.302760,-9.712860,10.324219
159,Face\0_10181.jpg,0.000000,-62.964367,-4.250488,-18.764114
195,Face\0_10223.jpg,5.005360,-67.648674,-2.867683,-13.315262
200,Face\0_1023.jpg,6.040335,-69.518360,-19.516724,-8.726837
...,...,...,...,...,...
57986,Face\1_8828_cloth.jpg,0.000000,67.450500,-7.608101,-18.461044
58007,Face\1_8995_surgical_green.jpg,0.000000,-70.780050,4.659103,-13.279739
58053,Face\1_9425_surgical_blue.jpg,0.000000,-63.907845,1.941788,6.072762
58113,Face\1_9990_cloth.jpg,0.000000,-67.614620,3.081459,-14.104141


torch.cuda.is_available()

In [28]:
# file1names = set(list(file['Name']))
# file2names = set(list(highPose['Name']))
# com = list(file2names.intersection(file1names))
# print(len(com))
# remains = list(file1names-set(com))
# len(remains)
# np.savetxt("C:\\Users\Girish\\Desktop\\FF-Projects\\Datasets\\augment5Thresholded.txt", list(highPose['Name']), fmt='%s')

In [21]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# highYaw = os.makedirs("C:\\Users\\Girish\\Desktop\\FF-Projects\\deep-head-pose\\output\\highYaw", exist_ok=True)
# os.makedirs("C:\\Users\\Girish\\Desktop\\FF-Projects\\deep-head-pose\\output\\highPose", exist_ok=True)
dest_dir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\augment5"
data_dir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\obscurredMask\\toAdd\\sideFace"
for name in highPose['Name']:
    path = os.path.split(name)
    # print(path)
    if path[0]=="Face":
        sourcePath = os.path.join(dest_dir, name)
        destinationPath = os.path.join(data_dir, path[-1])
        # print(sourcePath)
        # print(destinationPath)
        # break
        shutil.copy(sourcePath, destinationPath)
        # print(path[-1])
        # plt.imshow(Image.open(sourcePath))
        # plt.show()

In [ ]:
# import os
# import pandas as pd
# import shutil

# files = pd.read_csv("C:\\Users\\Girish\\Downloads\\meta.txt", sep=' ', header=None)
# files.head()

# file = files[files[1]==1].reset_index(drop=True)
# file.head()

# # # # highYaw = os.makedirs("C:\\Users\\Girish\\Desktop\\FF-Projects\\deep-head-pose\\output\\highYaw", exist_ok=True)
# # os.makedirs("C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\MisFIts", exist_ok=True)
# dest_dir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\MeGlass_120x120\\Eyeglasses"
# data_dir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\MeGlass_120x120\\MeGlass_120x120"
# for idx, rows in file.iterrows():
#     sourcePath = os.path.join(data_dir,rows[0])
#     destinationPath = os.path.join(dest_dir, rows[0])
#     # print(sourcePath, "\n", destinationPath)
#     # break
#     if os.path.exists(sourcePath):
#         print("done")
#         shutil.move(sourcePath, destinationPath)
#     else:
#         print(sourcePath)

# Accuracy Comparison

In [3]:
import os
import pandas as pd
import numpy as np

file = pd.read_csv("C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\DemoTestMnM.csv").sort_values(by='Name').reset_index(drop=True)
file

,Image,Name,isMaskedFace,M_Pass,isNonMaskedFace,NM_Pass,Mask@0.4,Mask@0.5,Mask@0.6,Mask@0.44
0,.//Mask//0_frame_0000.jpg,0_frame_0000.jpg,0.61,1,0.39,1,1,1,1,1
1,.//Nonmask//0_frame_0001.jpg,0_frame_0001.jpg,0.50,1,0.50,1,1,0,0,1
2,.//Nonmask//0_frame_0002.jpg,0_frame_0002.jpg,0.49,1,0.51,1,1,0,0,1
3,.//Nonmask//0_frame_0004.jpg,0_frame_0004.jpg,0.24,0,0.76,1,0,0,0,0
4,.//Nonmask//0_frame_0005.jpg,0_frame_0005.jpg,0.24,0,0.76,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2142,.//Nonmask//obscr_Betsy_McCaughey-101.jpg,obscr_Betsy_McCaughey-101.jpg,0.38,1,0.62,1,0,0,0,0
2143,.//Nonmask//obscr_Betty_Buckley-1.jpg,obscr_Betty_Buckley-1.jpg,0.58,1,0.42,1,1,1,0,1
2144,.//Nonmask//obscr_Betty_Buckley-10.jpg,obscr_Betty_Buckley-10.jpg,0.54,1,0.46,1,1,1,0,1
2145,.//Nonmask//obscr_Betty_Carter-10.jpg,obscr_Betty_Carter-10.jpg,0.46,1,0.54,1,1,0,0,1


In [6]:
file2 = pd.read_csv("C:\\Users\\Girish\\Desktop\\Project2_faceVSnonface\\Results_MaskNonmask\\MobileNetv3s_MaskNonmask_NF3\\predictions.csv",
                     names=['folder', 'Name', 'Pred_label'], header=0).sort_values(by='Name').reset_index(drop=True)
file2
# file2

,folder,Name,Pred_label
0,Mask,0_frame_0000.jpg,0
1,Nonmask,0_frame_0001.jpg,1
2,Nonmask,0_frame_0002.jpg,1
3,Nonmask,0_frame_0003.jpg,1
4,Nonmask,0_frame_0004.jpg,1
...,...,...,...
2606,Nonmask,obscr_Betsy_McCaughey-101.jpg,1
2607,Nonmask,obscr_Betty_Buckley-1.jpg,1
2608,Nonmask,obscr_Betty_Buckley-10.jpg,1
2609,Nonmask,obscr_Betty_Carter-10.jpg,1


In [7]:
file1names = set(list(file['Name']))
file2names = set(list(file2['Name']))
com = list(file1names.intersection(file1names))
len(com)

2147

In [8]:
subset_df = pd.merge(file2, file, how='inner', on='Name')
subset_df

,folder,Name,Pred_label,Image,isMaskedFace,M_Pass,isNonMaskedFace,NM_Pass,Mask@0.4,Mask@0.5,Mask@0.6,Mask@0.44
0,Mask,0_frame_0000.jpg,0,.//Mask//0_frame_0000.jpg,0.61,1,0.39,1,1,1,1,1
1,Nonmask,0_frame_0001.jpg,1,.//Nonmask//0_frame_0001.jpg,0.50,1,0.50,1,1,0,0,1
2,Nonmask,0_frame_0002.jpg,1,.//Nonmask//0_frame_0002.jpg,0.49,1,0.51,1,1,0,0,1
3,Nonmask,0_frame_0004.jpg,1,.//Nonmask//0_frame_0004.jpg,0.24,0,0.76,1,0,0,0,0
4,Nonmask,0_frame_0005.jpg,1,.//Nonmask//0_frame_0005.jpg,0.24,0,0.76,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2142,Nonmask,obscr_Betsy_McCaughey-101.jpg,1,.//Nonmask//obscr_Betsy_McCaughey-101.jpg,0.38,1,0.62,1,0,0,0,0
2143,Nonmask,obscr_Betty_Buckley-1.jpg,1,.//Nonmask//obscr_Betty_Buckley-1.jpg,0.58,1,0.42,1,1,1,0,1
2144,Nonmask,obscr_Betty_Buckley-10.jpg,1,.//Nonmask//obscr_Betty_Buckley-10.jpg,0.54,1,0.46,1,1,1,0,1
2145,Nonmask,obscr_Betty_Carter-10.jpg,1,.//Nonmask//obscr_Betty_Carter-10.jpg,0.46,1,0.54,1,1,0,0,1


In [9]:
numNonface, numFace = 0, 0
trueNF, falseNF, trueF, falseF = 0, 0, 0, 0
for idx, row in file.iterrows():
    true_class = os.path.split(row['Image'])[0].split("//")[-1]
    # print(row['Pass'])
    if true_class=="Nonmask":
        numNonface +=1
        if row['M_Pass']==0:
            trueNF += 1
        else: falseNF += 1

    elif true_class=="Mask":
        numFace += 1
        if row['M_Pass']==1:
            trueF += 1
        else: falseF+=1


print(f"From {numFace} Mask images {trueF} predicted as Mask and {falseF} are predicted as Nonmask")
print(f"From {numNonface} Nonmask images {trueNF} predicted as Nonmask and {falseNF} are predicted as Mask")
print(f"Nonmask classifier accuracy = {(trueNF+trueF)/(numFace+numNonface)}")
print("#"*50)
print(f"Accuracy for Mask = {trueF/numFace}")
print(f"Precision for Mask = {trueF/(trueF+falseNF)}")
print(f"Recall for Mask = {trueF/(trueF+falseF)}")
print("#"*50)
print(f"Accuracy for NonMask = {trueNF/numNonface}")
print(f"Precision for NonMask = {trueNF/(trueNF+falseF)}")
print(f"Recall for NonMask = {trueNF/(trueNF+falseNF)}")

From 147 Mask images 144 predicted as Mask and 3 are predicted as Nonmask
From 2000 Nonmask images 594 predicted as Nonmask and 1406 are predicted as Mask
Nonmask classifier accuracy = 0.34373544480670704
##################################################
Accuracy for Mask = 0.9795918367346939
Precision for Mask = 0.09290322580645162
Recall for Mask = 0.9795918367346939
##################################################
Accuracy for NonMask = 0.297
Precision for NonMask = 0.9949748743718593
Recall for NonMask = 0.297


In [10]:
numNonface, numFace = 0, 0
trueNF, falseNF, trueF, falseF = 0, 0, 0, 0
for idx, row in subset_df.iterrows():
    true_class = os.path.split(row['Image'])[0].split("//")[-1]
    # print(row['Pass'])
    if true_class=="Nonmask":
        numNonface +=1
        if row['Pred_label']==1:
            trueNF += 1
        else: falseNF += 1

    elif true_class=="Mask":
        numFace += 1
        if row['Pred_label']==0:
            trueF += 1
        else: falseF+=1


print(f"From {numFace} Mask images {trueF} predicted as Mask and {falseF} are predicted as Nonmask")
print(f"From {numNonface} Nonmask images {trueNF} predicted as Nonmask and {falseNF} are predicted as Mask")
print(f"Nonmask classifier accuracy = {(trueNF+trueF)/(numFace+numNonface)}")
print("#"*50)
print(f"Accuracy for Mask = {trueF/numFace}")
print(f"Precision for Mask = {trueF/(trueF+falseNF)}")
print(f"Recall for Mask = {trueF/(trueF+falseF)}")
print("#"*50)
print(f"Accuracy for NonMask = {trueNF/numNonface}")
print(f"Precision for NonMask = {trueNF/(trueNF+falseF)}")
print(f"Recall for NonMask = {trueNF/(trueNF+falseNF)}")

From 147 Mask images 135 predicted as Mask and 12 are predicted as Nonmask
From 2000 Nonmask images 1964 predicted as Nonmask and 36 are predicted as Mask
Nonmask classifier accuracy = 0.9776432231020028
##################################################
Accuracy for Mask = 0.9183673469387755
Precision for Mask = 0.7894736842105263
Recall for Mask = 0.9183673469387755
##################################################
Accuracy for NonMask = 0.982
Precision for NonMask = 0.9939271255060729
Recall for NonMask = 0.982


In [11]:
numNonface, numFace = 0, 0
trueNF, falseNF, trueF, falseF = 0, 0, 0, 0
for idx, row in subset_df.iterrows():
    true_class = os.path.split(row['Image'])[0].split("//")[-1]
    # print(row['Pass'])
    if true_class=="Nonmask":
        numNonface +=1
        if row['Mask@0.44']==0:
            trueNF += 1
        else: falseNF += 1

    elif true_class=="Mask":
        numFace += 1
        if row['Mask@0.44']==1:
            trueF += 1
        else: falseF+=1


print(f"From {numFace} Mask images {trueF} predicted as Mask and {falseF} are predicted as Nonmask")
print(f"From {numNonface} Nonmask images {trueNF} predicted as Nonmask and {falseNF} are predicted as Mask")
print(f"Nonmask classifier accuracy = {(trueNF+trueF)/(numFace+numNonface)}")
print("#"*50)
print(f"Accuracy for Mask = {trueF/numFace}")
print(f"Precision for Mask = {trueF/(trueF+falseNF)}")
print(f"Recall for Mask = {trueF/(trueF+falseF)}")
print("#"*50)
print(f"Accuracy for NonMask = {trueNF/numNonface}")
print(f"Precision for NonMask = {trueNF/(trueNF+falseF)}")
print(f"Recall for NonMask = {trueNF/(trueNF+falseNF)}")

From 147 Mask images 118 predicted as Mask and 29 are predicted as Nonmask
From 2000 Nonmask images 1818 predicted as Nonmask and 182 are predicted as Mask
Nonmask classifier accuracy = 0.9017233348858873
##################################################
Accuracy for Mask = 0.8027210884353742
Precision for Mask = 0.3933333333333333
Recall for Mask = 0.8027210884353742
##################################################
Accuracy for NonMask = 0.909
Precision for NonMask = 0.9842988630211154
Recall for NonMask = 0.909


In [ ]:
import os
import pandas as pd 
import numpy as np

file_dir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\test3"
csv_file = pd.read_csv("C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\test3_30.txt")
for id, row in csv_file.iterrows():
    className = os.path.split(row.iloc[0])[0]
    print(className)
    break
csv_file

# Misc

In [3]:
import os
import shutil
direc = r"C:\Users\Girish\Desktop\FF-Projects\MatchScoreAnalysis\NexDataCropped\femalecaucassian01719"
idx = 0
for file in os.listdir(direc):
    if file.endswith(('.jpg', '.png', '.jpeg')):
        idx+=1
        print(file)
        splits = file.split('_')
        extnsn = file.split('.')[-1]
        # print(extnsn)
        # new_name = f"NF_{file}"
        new_name = f"FC01719_{idx:03d}.{extnsn}"
        print(new_name)
        os.rename(os.path.join(direc,file), os.path.join(direc,new_name))
    # break

femaleIndian0049_001.png
FC01719_001.png
femaleIndian0049_002.png
FC01719_002.png
femaleIndian0049_003.png
FC01719_003.png
femaleIndian0049_004.png
FC01719_004.png
femaleIndian0049_005.png
FC01719_005.png
femaleIndian0049_006.png
FC01719_006.png
femaleIndian0049_007.png
FC01719_007.png
femaleIndian0049_008.png
FC01719_008.png
femaleIndian0049_009.png
FC01719_009.png
femaleIndian0049_010.png
FC01719_010.png
femaleIndian0049_011.png
FC01719_011.png
femaleIndian0049_012.png
FC01719_012.png
femaleIndian0049_013.png
FC01719_013.png
femaleIndian0049_014.png
FC01719_014.png
femaleIndian0049_015.png
FC01719_015.png
femaleIndian0049_016.png
FC01719_016.png
femaleIndian0049_017.png
FC01719_017.png
femaleIndian0049_018.png
FC01719_018.png
femaleIndian0049_019.png
FC01719_019.png
femaleIndian0049_020.png
FC01719_020.png
femaleIndian0049_021.png
FC01719_021.png
femaleIndian0049_022.png
FC01719_022.png
femaleIndian0049_023.png
FC01719_023.png
femaleIndian0049_024.png
FC01719_024.png
femaleIndian0049

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import shutil
import matplotlib.pyplot as plt


# path = "C:\\Users\\Girish\\Desktop\\Segragated-data"
path = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\test3"
folders = os.listdir(path)
print(folders)
# label = folders.index("Nonface")
# print(label)
data = []

filePath = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\MultilabelData.txt"

for folder in folders:
    for files in os.listdir(os.path.join(path, folder)):
        if files.endswith(('jpg', 'png', 'jpeg')):
            img_path = os.path.join(path, folder, files)
            print(img_path)
            img_name = os.path.split(img_path)[-1]
            img = Image.open(img_path)
            plt.imshow(img)
            plt.show(block=False)
            label = input("Give class label for the image (Nonface|Mask|Sunglass|Hat): ")
            plt.close()
            data.append([img_path, label])

np.savetxt(filePath, data, fmt='%s')

In [33]:
# import os
# import shutil
# from PIL import Image
# import cv2
# direc = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\obscurredMask\\toAdd\\sideFace"
# i=0
# for file in os.listdir(direc):
#     print(file)
#     # image = cv2.imread(os.path.join(direc,file))
#     image = Image.open(os.path.join(direc, file)).convert("RGB")
#     height = image.height
#     print(height)
#     if height==132:
#         os.remove(os.path.join(direc, file))

In [35]:
# import os
# import shutil
# from PIL import Image
# import cv2
# direc = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\obscurredMask\\Training\\Nonmask"
# text_file_path = "C:\\Users\\Girish\\Desktop\\Project2_faceVSnonface\\Results_MaskNonmask\\MobileNetv3L_MaskNonmask_7\\trainImageList.txt"
# with open(text_file_path, 'w') as file:
#     for filename in os.listdir(direc):
#         if filename.endswith(("jpg", "png", "jpeg", "tiff")):
#             file.write(os.path.join(direc,filename))
# file.close()

In [7]:
# import pandas as pd 
# # file = pd.read_table("C:\\Users\\Girish\\Desktop\\FF-Projects\\CR-FIQA\\fiqatest.txt")
# # print(list(file))

# with open("C:\\Users\\Girish\\Desktop\\FF-Projects\\CR-FIQA\\fiqatest.txt") as f:
#     absolute_list=f.readlines()
#     print(absolute_list[0].rstrip())

0landmk_03rahane.jpg


In [33]:
import os
import shutil

srcDir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\Ruo\\IMFD\\00000"
dstDir = "C:\\Users\\Girish\\Desktop\\FF-Projects\\Datasets\\obscurredMask\\toAdd\\mask"
count = 0
for file in os.listdir(srcDir):
    shutil.copy(os.path.join(srcDir, file), os.path.join(dstDir, file))
    count+=1
    if count == 200: break

In [1]:
import os
import numpy as np

path_dir = r"C:\Users\Girish\Desktop\FF-Projects\Datasets\Ruo\Multi-PIE-Sample\Images"
text_file_path = "C:\\Users\\Girish\\Desktop\\FF-Projects\\deep-head-pose\\Multi-PIE_samples.txt"

In [2]:
def list_folders_and_subfolders_and_print_file_sizes(directory):
  with open(text_file_path, 'w') as f:
    for root, dirs, files in os.walk(directory):
      for file in files:
        # if file.endswith(('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')):
          # print(os.path.join(root, file))
          f.write(f"{os.path.join(root, file)}\n")
  f.close()
list_folders_and_subfolders_and_print_file_sizes(path_dir)

In [10]:
'''
TO SELECT ONE SINGLE IMAGE FROM ALL IDENTITES PRESENT IN NEXDATA 
'''

import os
import shutil

fileDir = r"C:\Users\Girish\Desktop\FF-Projects\Datasets\Ruo\cropped_f3"
destPath = r"C:\Users\Girish\Desktop\FF-Projects\Datasets\NexData1only_2"
for folder in os.listdir(fileDir):
    os.makedirs(os.path.join(destPath, folder), exist_ok=True)
    for file in os.listdir(os.path.join(fileDir, folder)):
        if "C1" in set(file.split('-')):
            shutil.copy(os.path.join(fileDir, folder, file), os.path.join(destPath, folder, file))
